In [6]:
import pandas as pd
import seaborn as sns
import xgboost as xgb
import numpy as np
from dsgtools.metrics import ks
import pyreadr as pyr
import pyarrow.parquet as pq

In [7]:
from dsgtools.reporting import ks_tables

In [8]:
import sys
print(sys.version)

3.8.10 (default, May 19 2021, 13:12:57) [MSC v.1916 64 bit (AMD64)]


In [9]:
xgb.__version__

'1.4.2'

In [ ]:
file_dir = "E:/Projects/Phone Shell/Phone_Zone/PHZONE_Research/May_Machine_ReDials (PHZONE-911)/files"
df = pyr.read_r(file_dir + "/may_eqp1_inq1_mkiv.RDS")

In [ ]:
df = df[None]
df.to_parquet("may21_eqp1_inq1_mkiv_3_1_3.parquet")

In [10]:
df = pd.read_parquet('may21_eqp1_inq1_mkiv_3_1_3.parquet')

In [11]:
df = df.astype({"acctno": str, "gathered_ph": str})

In [12]:
dial = pd.read_csv("Q:/GPA_Phone_Testing_Results/PHONE TEST RESULTS (Accutrac)/2021 SUMMARY RESULTS/202105 May (Online and Batch)/Normalized Dial Results/full_data-May21.csv").dropna(subset = ['phone', 'rpc_status'])
dial = dial.astype({"account": str, "phone": str}) # change account and phone to strings
dial['phone'] = dial['phone'].str[0:10] # trim phone so that there is no .0 at the end
all_cols = list(dial.columns.values.tolist())
ranking_cols = all_cols[4:] # get all rank columns
for col in ranking_cols:
   dial[col] = dial[col].apply(lambda x: int(x) if x == x else "")  # change all rank cols to an integer and leave blank if not

cond = [
    (dial["rpc_status"].isin(['FAX', 'LEAD PHONE', 'DECEASED'])) # create condition of being an "Other" phone
]
choices = ["OTHER"]
dial["rpc_status"] = np.select(cond, choices, default = dial["rpc_status"]) 
valid_status = ["MACHINE", "WRONG PARTY", "VERIFIED", "NO ANSWER", "DISC.", "OTHER"] # Only keep valid status phones
dial = dial[dial['rpc_status'].isin(valid_status)]

dial

,account,phone,rpc_status,switch_type,pos-prod_3_0,pos-prod_3_0-cut_387,pos-prod_3_0_iq,pos-prod_3_0_iq-cut_387,pos-prod_3_0_eqp,pos-prod_3_0_eqp-cut_387,...,pos-online_cp_1,pos-online_pf_1,pos-online_below_the_line-inq_0,pos-online_below_the_line-inq_1,pos-ps30-dev-base,pos-ps30-dev-nooverwrite,pos-ps30-dev-yesoverwrite,pos-ps30-dev-tu,pos-_rnd_0,pos-_rnd_1
1,2105010001,2156596756,MACHINE,POTS,,,,,5,5,...,,,,,,,,,,5
2,2105010001,5712680885,WRONG PARTY,CELL,,,5,5,,,...,,,,,,,5,,3,
3,2105010001,5712742745,MACHINE,CELL,,,2,2,,,...,,,,,2,2,2,2,4,
4,2105010001,5712745572,VERIFIED,CELL,1,1,1,1,1,1,...,,,,,1,1,1,1,,
5,2105010001,5712763898,MACHINE,CELL,3,3,4,4,3,3,...,,,,,5,5,4,5,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151591,2105053000,6158042087,WRONG PARTY,CELL,4,4,,,4,4,...,,,,,5,5,3,5,,
151592,2105053000,6158244176,DISC.,POTS,,,,,,,...,,,,,,,,,,3
151593,2105053000,6158267338,WRONG PARTY,BLANK,,,,,,,...,,,,,,,,,,
151594,2105053000,6159361440,MACHINE,POTS,,,,,,,...,,,,,,,,,5,


In [13]:
dial_slim = dial[["account", "phone", "rpc_status"]]
dial_slim = dial_slim.rename(columns={'account': 'acctno', 'phone': 'gathered_ph'})

In [14]:
df.shape

(194667, 892)

In [15]:
dial_slim.shape

(141484, 3)

In [16]:
dial_slim

,acctno,gathered_ph,rpc_status
1,2105010001,2156596756,MACHINE
2,2105010001,5712680885,WRONG PARTY
3,2105010001,5712742745,MACHINE
4,2105010001,5712745572,VERIFIED
5,2105010001,5712763898,MACHINE
...,...,...,...
151591,2105053000,6158042087,WRONG PARTY
151592,2105053000,6158244176,DISC.
151593,2105053000,6158267338,WRONG PARTY
151594,2105053000,6159361440,MACHINE


In [17]:
df_merged = df.merge(dial_slim, 
                     how = 'inner', 
                     left_on = ["acctno", "gathered_ph"], 
                     right_on = ["acctno", "gathered_ph"])

In [18]:
df_merged

,psversion,acctno,in_fname,in_streetaddress,in_city,in_state,in_zipcode,in_dob,in_ssn,in_ph10,...,md_otp_count_180_day,md_otp_count_365_day,md_otp_count_730_day,mpp_listing_name_match,mpp_state_match,mpp_rp_state_match,mpp_src_SE_lseen_mr_flag,md_good_recent_act,mpp_src_ldate_bad_mth,rpc_status
0,31,2105010001,ANA,1560 MOORINGS DR UNIT 7B,RESTON,VA,201904268,,9,,...,0.0,0.0,0.0,1.0,1.0,NaN,2.0,0.0,36.0,DISC.
1,31,2105010001,ANA,1560 MOORINGS DR UNIT 7B,RESTON,VA,201904268,,9,,...,0.0,0.0,0.0,1.0,1.0,NaN,3.0,0.0,34.0,DISC.
2,31,2105010001,ANA,1560 MOORINGS DR UNIT 7B,RESTON,VA,201904268,,9,,...,2.0,2.0,2.0,1.0,1.0,NaN,2.0,1.0,NaN,VERIFIED
3,31,2105010001,ANA,1560 MOORINGS DR UNIT 7B,RESTON,VA,201904268,,9,,...,0.0,0.0,0.0,1.0,1.0,NaN,1.0,1.0,NaN,MACHINE
4,31,2105010001,ANA,1560 MOORINGS DR UNIT 7B,RESTON,VA,201904268,,9,,...,0.0,0.0,0.0,0.0,1.0,NaN,2.0,1.0,187.0,DISC.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118799,31,2105053000,SUE,771 CAROLYN LN,GALLATIN,TN,370664809,,9,,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,MACHINE
118800,31,2105053000,SUE,771 CAROLYN LN,GALLATIN,TN,370664809,,9,,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,DISC.
118801,31,2105053000,SUE,771 CAROLYN LN,GALLATIN,TN,370664809,,9,,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,MACHINE
118802,31,2105053000,SUE,771 CAROLYN LN,GALLATIN,TN,370664809,,9,,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,WRONG PARTY


In [19]:
df_merged.loc[df_merged["rpc_status"] == "VERIFIED", "dv_VERIFIED"] = 1
df_merged.loc[df_merged["rpc_status"] != "VERIFIED", "dv_VERIFIED"] = 0
df_merged.loc[df_merged["rpc_status"] == "DISC.", "dv_DISC"] = 1
df_merged.loc[df_merged["rpc_status"] != "DISC.", "dv_DISC"] = 0
df_merged.loc[df_merged["rpc_status"] == "WRONG PARTY", "dv_WRONG"] = 1
df_merged.loc[df_merged["rpc_status"] != "WRONG PARTY", "dv_WRONG"] = 0
df = df_merged

In [20]:
unq_accts = df["acctno"].unique()
unq_accts

array(['2105010001', '2105010002', '2105010004', ..., '2105052998',
       '2105052999', '2105053000'], dtype=object)

In [21]:
import random
random.seed(123)

In [22]:
dev = random.sample(list(unq_accts), k = round(0.7*len(unq_accts)))
len(dev)

6990

In [23]:
train = random.sample(dev, k = round(0.7*len(dev)))
len(train)

4893

In [24]:
test = list(set(dev) - set(train))
len(test)

2097

In [25]:
holdout = list(set(unq_accts) - set(dev))
len(holdout)

2996

In [26]:
import pickle
import os
os.getcwd()

'E:\\Useful Code\\Python\\XGBoost'

In [27]:
os.chdir("E:/Useful Code/Python/XGBoost")

In [28]:
with open(os.getcwd() + "/account_splits/train.txt", "wb") as fp:
    pickle.dump(train, fp)

with open(os.getcwd() + "/account_splits/test.txt", "wb") as fp:
    pickle.dump(test, fp)

with open(os.getcwd() + "/account_splits/holdout.txt", "wb") as fp:
    pickle.dump(holdout, fp)

In [29]:
with open(os.getcwd() + "/account_splits/train.txt", "rb") as fp:
    train = pickle.load(fp)

with open(os.getcwd() + "/account_splits/test.txt", "rb") as fp:
    test = pickle.load(fp)

with open(os.getcwd() + "/account_splits/holdout.txt", "rb") as fp:
    holdout = pickle.load(fp)

In [30]:
from dsgtools.eda import col_summary
cs = col_summary(df)
cs

C:\Users\OlsoBr02\Miniconda3\envs\data-science\lib\site-packages\dsgtools\eda\utils.py:92: RuntimeWarning: Degrees of freedom <= 0 for slice.
  variance = np.nanvar(x) if is_numeric_dtype(x) else pd.NA


,var,dtype,n_levels,levels,n_NA,pct_NA,variance
0,psversion,object,1,[31],0,0.000000,<NA>
1,acctno,object,9986,"[2105010001, 2105010002, 2105010004, 210501000...",0,0.000000,<NA>
2,in_fname,object,2701,"[ANA, BEATRICE, LEAH, NANCY, PAUL, CAROL, CRAI...",0,0.000000,<NA>
3,in_streetaddress,object,9966,"[1560 MOORINGS DR UNIT 7B, 1812 E LAFAYETTE ST...",0,0.000000,<NA>
4,in_city,object,3377,"[RESTON, STOCKTON, LAMPASAS, TUCSON, TAKOMA PA...",0,0.000000,<NA>
...,...,...,...,...,...,...,...
891,mpp_src_ldate_bad_mth,float64,264,<NA>,102343,0.861444,7634.557505
892,rpc_status,object,6,"[DISC., VERIFIED, MACHINE, NO ANSWER, WRONG PA...",0,0.000000,<NA>
893,dv_VERIFIED,float64,2,<NA>,0,0.000000,0.034517
894,dv_DISC,float64,2,<NA>,0,0.000000,0.215309


In [31]:
cs_char = cs[cs['variance'].isna()] # Get all character columns
cs_char = cs_char['var'].tolist()
cs_no_var = cs[cs['variance'] < 0.00001] # Get all columns with no variance
cs_no_var = cs_no_var['var'].tolist()

In [32]:
# Get Keep Names
dont_use_vars = [
  "p_phone_match_code_cnt",
  "p_Phone_Match_Level",
  "p_src_list_PPTH",
  "p_src_list_fdate_PPTH_mth",
  "p_src_list_ldate_PPTH_mth",
  "p_src_list_tof_PPTH_mth"
]

drop_vars = [
  "acctno",
  "LexID",
  "LexID_deceased",
  "LexID_deceased_date",
  "truedid",
  "subject_ssn_mismatch",
  "acctno_phone",
  "in_expgw_enabled",
  "in_targusgw_enabled",
  "in_tugw_enabled",
  "in_insgw_enabled",
  "in_fname",
  "in_streetaddress",
  "in_city",
  "in_state",
  "in_zipcode",
  "in_dob",
  "in_ssn",
  "in_ph10",
  "sysdate8",
  "archive_date",
  "gathered_ph",
  "dv_DISC",
  "dv_MACHINE",
  "dv_NO_ANSWER",
  "dv_VERIFIED",
  "dv_WRONG",
  "dv_DECEASED",
  "dv_FAX",
  "dv_LEAD_PHONE",
  "dv_NOT_ENOUGH_INFO",
  "dv_OTHER",
  "FileInd",
  "status",
  "dial_status",
  "Status_group",
  "Sample_YM",
  "Sample_Gateway",
  "Sample_Party",
  "key_acct",
  "key_acct_gateway",
  "key_acct_phone",
  "key_acct_gateway_phone",
  "Phone_Score", 
  "meta_archive_date",
  "md_carrier"
  #"Score_PS21_model06_2007"
]

untrust_vars = [
  "mpp_source",
  "mpp_type",
  "mpp_app_npa_effective_dt_mth"
]

In [33]:
all_drop_vars = np.unique(cs_char+cs_no_var+dont_use_vars+drop_vars+untrust_vars).tolist()
eda_drop_vars = [i for i in df.columns.tolist() if i.startswith('eda_')]
pp_drop_vars = [i for i in df.columns.tolist() if i.startswith('pp_')]
bur_drop_vars = [i for i in df.columns.tolist() if i.startswith('bureau_')]
inq_drop_vars = [i for i in df.columns.tolist() if i.startswith('inq_')]
phn_drop_vars = [i for i in df.columns.tolist() if i.startswith('phone_')]
ins_drop_vars = [i for i in df.columns.tolist() if i.startswith('internal_')]

final_drop_vars = np.unique(all_drop_vars+eda_drop_vars+pp_drop_vars+
                            bur_drop_vars+inq_drop_vars+phn_drop_vars+ins_drop_vars).tolist()
mod_attr_names = df.columns.difference(final_drop_vars).tolist()

In [34]:
# Save attribute names
with open(os.getcwd() + "/var_names/mod_attr_names.txt", "wb") as fp:
    pickle.dump(mod_attr_names, fp)
    
with open(os.getcwd() + "/var_names/mod_attr_names.txt", "rb") as fp:
    mod_attr_names = pickle.load(fp)

In [35]:
# Fill special values
df.fillna(-99999, inplace=True)

In [36]:
# Create Weights for Different Performance
weight_cond = [
    (df['dv_VERIFIED'] == 1),
    (df['dv_DISC'] == 1),
    (df['dv_WRONG'] == 1)
]
weight_val = [1, 1, 0.5] # Wrong Party with half weight

In [37]:
df['weight'] = np.select(weight_cond, weight_val)

In [38]:
weights = df['weight'].astype(np.float32)

In [39]:
# Create Train/Val Split Conditions
split_cond = [
    (df['acctno'].isin(train)),
    (df['acctno'].isin(test)),
    (df['acctno'].isin(holdout)) 
]
split_val = ['train', 'test', 'holdout']

In [40]:
df['split'] = np.select(split_cond, split_val)

In [41]:
dev = (df["split"] == "train").to_numpy()
tst = (df["split"] == "test").to_numpy()
val = (df["split"] == "holdout").to_numpy()

In [42]:
X_All = df.loc[:,mod_attr_names]
y = df["dv_VERIFIED"]

df_train = df.loc[dev,["dv_VERIFIED", "dv_DISC","dv_WRONG","rpc_status","acctno", "weight"]]
df_test = df.loc[tst,["dv_VERIFIED", "dv_DISC","dv_WRONG","rpc_status","acctno", "weight"]]

train_types = ['VERIFIED', 'DISC.', 'WRONG PARTY']
X_train = df.loc[dev,]
X_train = X_train[X_train['rpc_status'].isin(train_types)]
X_train_sub = X_train.loc[:,mod_attr_names]
X_train_weight = X_train.loc[:,['weight']].astype(np.float32)

X_test = df.loc[tst,]
X_test = X_test[X_test['rpc_status'].isin(train_types)]
X_test_sub = X_test.loc[:,mod_attr_names]
X_test_weight = X_test.loc[:,['weight']].astype(np.float32)

y_train = X_train.loc[:,['dv_VERIFIED']].astype(np.float32)
y_test = X_test.loc[:,['dv_VERIFIED']].astype(np.float32)

In [43]:
# Create xgb matrices for training
dtrain = xgb.DMatrix(data = X_train_sub, 
                     label = y_train,
                     weight = X_train_weight,
                     missing = -99999)
dtest = xgb.DMatrix(data = X_test_sub, 
                    label = y_test,
                    weight = X_test_weight,
                    missing = -99999)

In [44]:
# Set base parameters for phone model
params = dict(
    eta=0.1,
    max_depth=0,
    min_child_weight=25,
    subsample=0.99,
    tree_method="hist",
    grow_policy="lossguide",
    max_leaves=14,
    objective="binary:logitraw",
    eval_metric="auc",
    seed=0,
)
evallist = [(dtrain, "train"),(dtest, "test")]

In [45]:
base_mod = xgb.train(params = params, 
                     dtrain = dtrain,
                     num_boost_round = 1000,
                     evals = evallist,
                     early_stopping_rounds = 100,
                     verbose_eval = 50
                    )

[0]	train-auc:0.87022	test-auc:0.86616
[50]	train-auc:0.95307	test-auc:0.94125
[100]	train-auc:0.96473	test-auc:0.94479
[150]	train-auc:0.97130	test-auc:0.94472
[200]	train-auc:0.97591	test-auc:0.94427
[218]	train-auc:0.97735	test-auc:0.94395


In [46]:
# dsgtools way
from dsgtools.ensemble import glba

base_mod = glba.XGBClassifier(
    n_estimators=1_000, missing=-99999, use_label_encoder=False, **params
)

In [47]:
base_mod.fit(X_train_sub, 
             y_train, 
             sample_weight = X_train_weight,
             eval_set=[(X_test_sub, y_test)], 
             sample_weight_eval_set = [X_test_weight],
             eval_metric="auc",
             early_stopping_rounds=100,
             verbose=50, #number of iterations to output metric
            )

[0]	validation_0-auc:0.86616
[50]	validation_0-auc:0.94125
[100]	validation_0-auc:0.94479
[150]	validation_0-auc:0.94472
[200]	validation_0-auc:0.94427
[219]	validation_0-auc:0.94390


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.1,
              eval_metric='auc', gamma=0, gpu_id=-1, grow_policy='lossguide',
              importance_type='gain', interaction_constraints='',
              learning_rate=0.100000001, max_delta_step=0, max_depth=0,
              max_leaves=14, min_child_weight=25, missing=-99999,
              monotone_constraints='()', n_estimators=1000, n_jobs=8,
              num_parallel_tree=1, objective='binary:logitraw', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0,
              subsample=0.99, tree_method='hist', ...)

In [48]:
base_mod.save_model(os.getcwd() + "/models/base_mod.bin")
base_mod.save_model(os.getcwd() + "/models/base_mod_r.model")

In [49]:
base_mod = glba.XGBClassifier.load_model(os.getcwd() + "/models/base_mod.bin")

In [50]:
vpred_base = base_mod.predict(X_All.loc[val,mod_attr_names], output_margin=True)

In [51]:
ks_base = ks(y[val], vpred_base)
ks(y[val], vpred_base)

0.6064242090873058

In [52]:
df['vpred_base'] = base_mod.predict(X_All.loc[:,mod_attr_names], output_margin=True)

In [53]:
df['Sample_Party'] = df['acctno'].str[5:6]

In [54]:
ks_table_base = ks_tables(x="vpred_base",y="dv_VERIFIED", number_bins=100, df=df, groups="Sample_Party", reverse=True)

In [ ]:
print(ks_table_base.tables())

In [56]:
print(ks_table_base.summary())

        Score         Perf            By Value        KS       AUC      Gini  \
0  vpred_base  dv_VERIFIED       Overall  None -0.649589  0.101173 -0.797654   
1  vpred_base  dv_VERIFIED  Sample_Party     1 -0.597199  0.125234 -0.749532   
2  vpred_base  dv_VERIFIED  Sample_Party     3 -0.625329  0.113327 -0.773346   
3  vpred_base  dv_VERIFIED  Sample_Party     4 -0.674047  0.095368 -0.809264   
4  vpred_base  dv_VERIFIED  Sample_Party     5 -0.669096  0.093660 -0.812680   

          N    # 1s      # 0s   1s Rate   0s Rate       Min       Max  \
0  118804.0  4253.0  114551.0  0.035798  0.964202 -7.987835  5.155485   
1   30560.0   752.0   29808.0  0.024607  0.975393 -7.957120  4.251848   
2   30647.0   821.0   29826.0  0.026789  0.973211 -7.957570  4.058609   
3   27446.0  1354.0   26092.0  0.049333  0.950667 -7.987835  5.155485   
4   30151.0  1326.0   28825.0  0.043979  0.956021 -7.957186  4.798064   

   # Score Exceptions  # Missing Tag  
0                 0.0            0.0  
1 

In [57]:
ks_table_base.to_xlsx(filename=os.getcwd() + '/reports/base_model_ks_table.xlsx', 
                      overwrite=True)

In [58]:
# Capture Rate Model

base_mod_capt = glba.XGBClassifier(n_estimators=1_000, missing=-99999, 
                                   use_label_encoder=False,
                                   **params)

In [59]:
from dsgtools.ensemble.xgbutils import metric_to_xgb_eval_metric
from dsgtools.metrics import capture_rate

dof15_eval_metric = metric_to_xgb_eval_metric(capture_rate, maximize=True, dof=0.15)
callable(dof15_eval_metric)


True

In [60]:
base_mod_capt.fit(
    X_train_sub, 
    y_train, 
    sample_weight = X_train_weight,
    eval_set=[(X_test_sub, y_test)], 
    sample_weight_eval_set = [X_test_weight],
    eval_metric=dof15_eval_metric,
    early_stopping_rounds=100,
    verbose=50, #number of iterations to output metric
)

[0]	validation_0-auc:0.86616	validation_0-capture_rate:-0.37736
[50]	validation_0-auc:0.94125	validation_0-capture_rate:-0.47725
[100]	validation_0-auc:0.94479	validation_0-capture_rate:-0.47725
[144]	validation_0-auc:0.94477	validation_0-capture_rate:-0.47947


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.1,
              eval_metric='auc', gamma=0, gpu_id=-1, grow_policy='lossguide',
              importance_type='gain', interaction_constraints='',
              learning_rate=0.100000001, max_delta_step=0, max_depth=0,
              max_leaves=14, min_child_weight=25, missing=-99999,
              monotone_constraints='()', n_estimators=1000, n_jobs=8,
              num_parallel_tree=1, objective='binary:logitraw', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0,
              subsample=0.99, tree_method='hist', ...)

In [61]:
df['vpred_base_capt'] = base_mod_capt.predict(X_All.loc[:,mod_attr_names], output_margin=True)